# CRAM Hands-On

This is a Jupyter Notebook. It is usually used for Python code, but we installed a Common-Lisp kernel. Try it, click into the code block and hit CTRL-Enter or the icon in the header that looks like a *Play*-button.

In [1]:
"Welcome to the CRAM hands-on tutorial of the EASE Fall School 2022!"

"Welcome to the CRAM hands-on tutorial of the EASE Fall School 2022!"

This is Hello-World for Lisp, just a string that returns it's own value.

Over the course you can first check if your setup works, then there is an introduction to Lisp (lesson 1), basics of CRAM and the Bullet Physics simulator (lesson 2), writing a plan for a robot to pick & place objects (lesson 3) and finally, failure handling (lesson 4). Each lesson is it's own .ipynb file, you can see them listed on the left. All of your progress, every piece of code you write, will be saved to your local disk, so you can pick up where you left off whenever you want.

Jupyter Notebook consists of text blocks and code blocks. Code blocks have these `[ ]` brackets left of them. If they are empty, they haven't been executed yet. If they got a number, like `[42]` it has been executed. While they are executed, they show an asterisk `[*]`. You shouldn't execute code while another is still being executed. Some processes take a while to get done, e.g. loading the project into the kernel or starting the simulation environment. Just be patient. If it takes unreasonably long, hit the button that looks like *Stop* to kill a process that's still showing `[*]`, and hit it multiple times to kill all other running processes if you missed some. Sometimes it helps to just restart the kernel, or kill the `docker-compose` process from your terminal, outside of Jupyter. Keep in mind, that every time the Lisp kernel is restarted, any global parameters you defined in your last session need to be defined again by executing the code blocks.

Let's move on and check your setup.

## Setup

To check the setup, we are going to execute the pick & place demo for setting a breakfast table. First, make sure that x-forwarding is set up. CRAM is tightly bound to the Bullet Physics simulator, which runs with GLUT, an OpenGL wrapper. Without that simulator, we can't run our experiments.

### Linux
X-Forwarding on linux is easily enabled by executing the following in a new terminal on your host machine.
```bash
xhost +local:docker # install x11-xserver-utils if the command xhost can't be found
```
If it's stuck or can't open the display, check your `DISPLAY` variable with
```bash
echo $DISPLAY
```
and set it with
```bash
export DISPLAY=:0
```
or :1, depending on your host's display setup. The restart the `docker-compose up` process by killing it with CTRL-C twice. 

Btw, this `DISPLAY` got nothing to do with multiple displays connected to your PC, but the display manager. If you are running Wayland, which is the default for certain Debian distributions, we suggest to [switch to xorg](https://linuxconfig.org/how-to-enable-disable-wayland-on-ubuntu-20-04-desktop) instead. It's way more stable anyway.

### Windows
Remember to [configure and run VcXsrv](https://medium.com/javarevisited/using-wsl-2-with-x-server-linux-on-windows-a372263533c3) to open up the display for WSL. Watch out especially for the `-wgl` switch in the arguments for the shortcut of the executable. In the WSL image the `DISPLAY` variable is automatically set to the IP and display-name of the Windows display, and the Docker container inherits this variable. You can check it in the WSL bash terminal with
```bash
echo $DISPLAY
```
Execute the following to check if the Firewall settings are allowing communication between WSL and Windows
```bash
glxgears
```
It should show some moving gears. If they don't move, make sure you have VcXsrv installed, and not xMing, which can't handle GLUT applications well. Disable your Firewall for Public networks for a moment and try again, to verify that the issue is actually with the Firewall.


## Execute the pick & place demo

Now we get to the interesting part, this is the moment of truth. Loading the CRAM package will try to initialize the display for Bullet Physics. If this command fails, meaning, when the kernel dies, there's an issue with the display setup.

It will print a lot. You can hide the outprint by hitting the blue vertical bar left of the code block. Watch out for the `[*]` and wait for it to turn into an number, that's when the compiler is done.

In [ ]:
(asdf:load-system :cram-pr2-pick-place-demo)

If this worked, the display variables seem to be set up fine. Now we are booting up the system. The Bullet Physics windows should now pop up, load the environment and finally the robot.

In [ ]:
(roslisp-utilities:startup-ros)

The process is successful when Bullet Physics comes up, the process terminates, and you can move the camera in Bullet with your mouse (left and right mouse buttons).

When you reboot the Docker container, remember to execute `asdf:load-system` before `startup-ros`.

Troubleshoot:
* Usually, on the first boot, we experience that the Bullet Physics window comes up, but the code block doesn't terminate.
  * Stop the process via the `Stop` button at the top and execute the `startup-ros` again.
  * If that doesn't help, kill the `docker-compose up` process in our terminal and launch it again, aka 'have you tried turning it off and on again'.
* If the kernel stops, also try to restart docker-compose
* If the terminal, where docker-compose is running, says something about *fgConfig* and *missing Component*, it's a rendering issue, usually on Windows. Check your VcXsrv-shortcut arguments and make sure to use VcXsrv instead of xMing.

Finally, let's start the the demo. You can remove `'(:bowl)` if you want to see the demo with all objects.

In [ ]:
(urdf-proj:with-simulated-robot
 (demo::setting-demo '(:bowl)))